# Views and URL Routing

## Learning Objectives

By the end of this notebook, you will be able to:

1. Create function-based views (FBVs) to handle HTTP requests
2. Work with HttpRequest and HttpResponse objects
3. Configure URL patterns using path() and include()
4. Use URL parameters and converters
5. Create class-based views (CBVs) for common patterns
6. Use Django's generic views (ListView, DetailView, CreateView)
7. Handle redirects and errors properly

---

## 1. Introduction to Views

A **view** in Django is a Python function or class that:
1. Receives an HTTP request
2. Processes it (queries database, performs logic, etc.)
3. Returns an HTTP response

Views are the "V" in MVT (Model-View-Template) - they contain the business logic of your application.

### Request-Response Flow

```
Browser → URL Router → View → Template (optional) → Response → Browser
                         ↓
                       Model (optional)
```

## 2. Function-Based Views (FBVs)

The simplest type of view is a function that takes a request and returns a response:

```python
# catalog/views.py
from django.http import HttpResponse

def hello_world(request):
    """A simple view that returns a greeting."""
    return HttpResponse("Hello, World!")
```

### More Complete Example

```python
# catalog/views.py
from django.http import HttpResponse
from django.shortcuts import render
from .models import Book

def book_list(request):
    """Display a list of all books."""
    books = Book.objects.all()
    return render(request, 'catalog/book_list.html', {'books': books})


def book_detail(request, book_id):
    """Display details for a specific book."""
    book = Book.objects.get(pk=book_id)
    return render(request, 'catalog/book_detail.html', {'book': book})
```

## 3. HttpRequest Object

Every view receives an `HttpRequest` object as its first argument. It contains information about the request:

In [ ]:
# HttpRequest attributes and methods (for reference)

request_attributes = {
    # Request information
    'request.method': 'HTTP method (GET, POST, PUT, DELETE, etc.)',
    'request.path': 'URL path (e.g., /catalog/books/)',
    'request.GET': 'QueryDict of GET parameters (?name=value)',
    'request.POST': 'QueryDict of POST data (form submissions)',
    'request.FILES': 'Dict of uploaded files',
    'request.body': 'Raw HTTP request body (bytes)',
    
    # Headers
    'request.headers': 'Case-insensitive dict of headers',
    'request.content_type': 'MIME type of request body',
    
    # User and session
    'request.user': 'The logged-in user (or AnonymousUser)',
    'request.session': 'Session data dictionary',
    
    # Meta information
    'request.META': 'All HTTP headers and server info',
    'request.scheme': 'http or https',
    
    # Useful methods
    'request.is_ajax()': 'True if XMLHttpRequest (deprecated)',
    'request.is_secure()': 'True if HTTPS',
    'request.get_host()': 'Host/domain name',
    'request.get_full_path()': 'Path with query string',
}

print("HttpRequest Attributes and Methods:")
print("=" * 60)
for attr, desc in request_attributes.items():
    print(f"{attr:30} - {desc}")

### Working with Request Data

```python
# catalog/views.py
from django.http import HttpResponse

def search(request):
    """Handle search with GET parameters."""
    # URL: /search/?q=python&category=books
    
    # Get query parameter (with default)
    query = request.GET.get('q', '')  # '' if not provided
    category = request.GET.get('category', 'all')
    
    # Get list of values (for checkboxes)
    tags = request.GET.getlist('tags')  # Returns list
    
    return HttpResponse(f"Searching for: {query} in {category}")


def contact_form(request):
    """Handle form submission with POST data."""
    if request.method == 'POST':
        name = request.POST.get('name')
        email = request.POST.get('email')
        message = request.POST.get('message')
        
        # Process the form data...
        return HttpResponse("Thank you for your message!")
    
    # Show form for GET requests
    return render(request, 'catalog/contact.html')
```

## 4. HttpResponse Object

Views must return an `HttpResponse` object (or subclass):

```python
from django.http import (
    HttpResponse,
    HttpResponseRedirect,
    HttpResponseNotFound,
    HttpResponseForbidden,
    JsonResponse,
)

# Basic response
def simple_view(request):
    return HttpResponse("Hello, World!")

# Response with status code
def custom_status(request):
    return HttpResponse("Created!", status=201)

# Response with headers
def with_headers(request):
    response = HttpResponse("Content")
    response['X-Custom-Header'] = 'value'
    response['Content-Type'] = 'text/plain'
    return response

# JSON response
def api_view(request):
    data = {'name': 'Python', 'version': '3.11'}
    return JsonResponse(data)

# JSON response with list
def api_list(request):
    data = [1, 2, 3]
    return JsonResponse(data, safe=False)  # safe=False for non-dict

# Redirect
def redirect_view(request):
    return HttpResponseRedirect('/new-location/')

# 404 Not Found
def not_found_view(request):
    return HttpResponseNotFound("Page not found")
```

## 5. URL Configuration

URLs are configured using `urlpatterns` in `urls.py` files.

### Project-Level URLs

```python
# mysite/urls.py (project-level)
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('catalog/', include('catalog.urls')),  # Include app URLs
    path('blog/', include('blog.urls')),
    path('', include('home.urls')),  # Homepage
]
```

### App-Level URLs

```python
# catalog/urls.py (app-level)
from django.urls import path
from . import views

app_name = 'catalog'  # Namespace for URL reversing

urlpatterns = [
    path('', views.index, name='index'),
    path('books/', views.book_list, name='book_list'),
    path('books/<int:book_id>/', views.book_detail, name='book_detail'),
    path('search/', views.search, name='search'),
]
```

## 6. URL Parameters and Converters

Django can capture values from URLs and pass them to views:

```python
# catalog/urls.py
from django.urls import path
from . import views

urlpatterns = [
    # Basic integer parameter
    path('books/<int:book_id>/', views.book_detail, name='book_detail'),
    # Matches: /books/1/, /books/42/
    
    # String parameter
    path('author/<str:name>/', views.author_detail, name='author_detail'),
    # Matches: /author/hemingway/, /author/j-k-rowling/
    
    # Slug parameter (letters, numbers, hyphens, underscores)
    path('category/<slug:slug>/', views.category, name='category'),
    # Matches: /category/science-fiction/, /category/how_to/
    
    # UUID parameter
    path('order/<uuid:order_id>/', views.order_detail, name='order_detail'),
    # Matches: /order/075194d3-6885-417e-a8a8-6c931e272f00/
    
    # Path parameter (includes slashes)
    path('files/<path:file_path>/', views.download, name='download'),
    # Matches: /files/docs/2023/report.pdf/
]
```

### Built-in Path Converters

| Converter | Description | Example |
|-----------|-------------|--------|
| `str` | Any non-empty string (excludes `/`) | `hello-world` |
| `int` | Zero or positive integer | `42` |
| `slug` | ASCII letters, numbers, hyphens, underscores | `my-post-title` |
| `uuid` | UUID format | `075194d3-6885-...` |
| `path` | Any non-empty string (includes `/`) | `docs/2023/file.pdf` |

### Views with URL Parameters

```python
# catalog/views.py
from django.shortcuts import render, get_object_or_404
from .models import Book, Author, Category

def book_detail(request, book_id):
    """Display a single book."""
    # get_object_or_404 returns 404 if not found
    book = get_object_or_404(Book, pk=book_id)
    return render(request, 'catalog/book_detail.html', {'book': book})


def author_detail(request, name):
    """Display books by an author."""
    author = get_object_or_404(Author, slug=name)
    books = author.books.all()
    return render(request, 'catalog/author.html', {
        'author': author,
        'books': books
    })


def category(request, slug):
    """Display books in a category."""
    category = get_object_or_404(Category, slug=slug)
    books = Book.objects.filter(category=category)
    return render(request, 'catalog/category.html', {
        'category': category,
        'books': books
    })
```

## 7. URL Reversing

Instead of hardcoding URLs, use the `reverse()` function or `{% url %}` template tag:

```python
# In views
from django.urls import reverse
from django.http import HttpResponseRedirect

def create_book(request):
    # ... create book logic ...
    book = Book.objects.create(...)
    
    # Redirect to book detail page
    url = reverse('catalog:book_detail', args=[book.id])
    # url = '/catalog/books/5/'
    return HttpResponseRedirect(url)
    
    # Or with keyword arguments
    url = reverse('catalog:book_detail', kwargs={'book_id': book.id})


# In templates
# <a href="{% url 'catalog:book_detail' book.id %}">View Book</a>
# <a href="{% url 'catalog:book_list' %}">All Books</a>
```

## 8. Shortcuts

Django provides useful shortcut functions:

```python
from django.shortcuts import render, redirect, get_object_or_404, get_list_or_404

# render() - Render template with context
def book_list(request):
    books = Book.objects.all()
    return render(request, 'catalog/book_list.html', {'books': books})
    # Equivalent to:
    # template = loader.get_template('catalog/book_list.html')
    # return HttpResponse(template.render({'books': books}, request))


# redirect() - Redirect to URL or view name
def old_page(request):
    return redirect('catalog:book_list')  # By view name
    return redirect('/catalog/books/')     # By URL
    return redirect(book_object)           # By object with get_absolute_url()


# get_object_or_404() - Get object or raise 404
def book_detail(request, book_id):
    book = get_object_or_404(Book, pk=book_id)
    # Equivalent to:
    # try:
    #     book = Book.objects.get(pk=book_id)
    # except Book.DoesNotExist:
    #     raise Http404("Book not found")
    return render(request, 'catalog/book_detail.html', {'book': book})


# get_list_or_404() - Get list or raise 404 if empty
def author_books(request, author_id):
    books = get_list_or_404(Book, author_id=author_id)
    return render(request, 'catalog/book_list.html', {'books': books})
```

## 9. Class-Based Views (CBVs)

Class-based views organize code using object-oriented principles:

```python
# catalog/views.py
from django.views import View
from django.http import HttpResponse
from django.shortcuts import render

class BookListView(View):
    """Display a list of books."""
    
    def get(self, request):
        """Handle GET requests."""
        books = Book.objects.all()
        return render(request, 'catalog/book_list.html', {'books': books})
    
    def post(self, request):
        """Handle POST requests."""
        # Process form submission
        return HttpResponse("Book created!")


class BookDetailView(View):
    """Display a single book."""
    
    def get(self, request, book_id):
        book = get_object_or_404(Book, pk=book_id)
        return render(request, 'catalog/book_detail.html', {'book': book})
```

### URL Configuration for CBVs

```python
# catalog/urls.py
from django.urls import path
from .views import BookListView, BookDetailView

urlpatterns = [
    path('books/', BookListView.as_view(), name='book_list'),
    path('books/<int:book_id>/', BookDetailView.as_view(), name='book_detail'),
]
```

## 10. Generic Views

Django provides generic views for common patterns:

### ListView

```python
# catalog/views.py
from django.views.generic import ListView
from .models import Book

class BookListView(ListView):
    """Display a paginated list of books."""
    model = Book
    template_name = 'catalog/book_list.html'  # Default: book_list.html
    context_object_name = 'books'  # Default: object_list
    paginate_by = 10  # Enable pagination
    ordering = ['-published_date']  # Order by date descending
    
    def get_queryset(self):
        """Customize the queryset."""
        queryset = super().get_queryset()
        return queryset.filter(in_stock=True)
    
    def get_context_data(self, **kwargs):
        """Add extra context."""
        context = super().get_context_data(**kwargs)
        context['total_count'] = Book.objects.count()
        return context
```

### DetailView

```python
from django.views.generic import DetailView

class BookDetailView(DetailView):
    """Display a single book."""
    model = Book
    template_name = 'catalog/book_detail.html'  # Default: book_detail.html
    context_object_name = 'book'  # Default: object
    pk_url_kwarg = 'book_id'  # URL parameter name (default: 'pk')
    
    def get_context_data(self, **kwargs):
        """Add related books to context."""
        context = super().get_context_data(**kwargs)
        context['related_books'] = Book.objects.filter(
            author=self.object.author
        ).exclude(pk=self.object.pk)[:5]
        return context
```

### CreateView, UpdateView, DeleteView

```python
from django.views.generic import CreateView, UpdateView, DeleteView
from django.urls import reverse_lazy

class BookCreateView(CreateView):
    """Create a new book."""
    model = Book
    template_name = 'catalog/book_form.html'
    fields = ['title', 'author', 'isbn', 'price', 'published_date']
    success_url = reverse_lazy('catalog:book_list')
    
    def form_valid(self, form):
        """Called when valid form data is POSTed."""
        form.instance.created_by = self.request.user
        return super().form_valid(form)


class BookUpdateView(UpdateView):
    """Update an existing book."""
    model = Book
    template_name = 'catalog/book_form.html'
    fields = ['title', 'price', 'in_stock']
    pk_url_kwarg = 'book_id'
    
    def get_success_url(self):
        """Redirect to book detail after update."""
        return reverse_lazy('catalog:book_detail', kwargs={'book_id': self.object.pk})


class BookDeleteView(DeleteView):
    """Delete a book."""
    model = Book
    template_name = 'catalog/book_confirm_delete.html'
    success_url = reverse_lazy('catalog:book_list')
    pk_url_kwarg = 'book_id'
```

### URL Configuration

```python
# catalog/urls.py
urlpatterns = [
    path('books/', BookListView.as_view(), name='book_list'),
    path('books/<int:book_id>/', BookDetailView.as_view(), name='book_detail'),
    path('books/new/', BookCreateView.as_view(), name='book_create'),
    path('books/<int:book_id>/edit/', BookUpdateView.as_view(), name='book_update'),
    path('books/<int:book_id>/delete/', BookDeleteView.as_view(), name='book_delete'),
]
```

## 11. Redirects and Error Handling

### Redirects

```python
from django.shortcuts import redirect
from django.http import HttpResponseRedirect, HttpResponsePermanentRedirect
from django.urls import reverse

def old_view(request):
    # Temporary redirect (302)
    return redirect('catalog:book_list')

def moved_permanently(request):
    # Permanent redirect (301) - for SEO
    return HttpResponsePermanentRedirect('/new-url/')

def after_login(request):
    # Redirect to next URL or default
    next_url = request.GET.get('next', 'catalog:index')
    return redirect(next_url)
```

### Error Handling

```python
from django.http import Http404, HttpResponseForbidden, HttpResponseBadRequest

def book_detail(request, book_id):
    try:
        book = Book.objects.get(pk=book_id)
    except Book.DoesNotExist:
        raise Http404("Book not found")
    return render(request, 'catalog/book_detail.html', {'book': book})


def admin_only(request):
    if not request.user.is_staff:
        return HttpResponseForbidden("Access denied")
    return render(request, 'admin/dashboard.html')
```

### Custom Error Pages

Create templates for error pages:
- `templates/404.html` - Page not found
- `templates/500.html` - Server error
- `templates/403.html` - Forbidden
- `templates/400.html` - Bad request

```python
# mysite/urls.py (at the end)
handler404 = 'catalog.views.custom_404'
handler500 = 'catalog.views.custom_500'

# catalog/views.py
def custom_404(request, exception):
    return render(request, '404.html', status=404)

def custom_500(request):
    return render(request, '500.html', status=500)
```

## 12. View Decorators

Decorators add functionality to views:

```python
from django.views.decorators.http import require_http_methods, require_GET, require_POST
from django.contrib.auth.decorators import login_required, permission_required
from django.views.decorators.cache import cache_page

# Require specific HTTP methods
@require_GET
def book_list(request):
    return render(request, 'catalog/book_list.html')

@require_POST
def create_book(request):
    # Only accepts POST requests
    pass

@require_http_methods(['GET', 'POST'])
def book_form(request):
    pass

# Require login
@login_required
def my_books(request):
    books = Book.objects.filter(owner=request.user)
    return render(request, 'catalog/my_books.html', {'books': books})

@login_required(login_url='/accounts/login/')
def profile(request):
    pass

# Require permission
@permission_required('catalog.add_book')
def add_book(request):
    pass

# Cache the response
@cache_page(60 * 15)  # Cache for 15 minutes
def expensive_view(request):
    pass
```

---

## Exercises

### Exercise 1: Basic Function-Based View

Create a view that displays "Welcome to our bookstore! We have X books available." where X is the count of books in stock.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.http import HttpResponse
from .models import Book

def welcome(request):
    """Display welcome message with book count."""
    book_count = Book.objects.filter(in_stock=True).count()
    return HttpResponse(f"Welcome to our bookstore! We have {book_count} books available.")


# catalog/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.welcome, name='welcome'),
]
```

</details>

### Exercise 2: URL Parameters

Create a view that accepts a category slug and displays all books in that category. If no books exist, return a 404.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.shortcuts import render, get_object_or_404, get_list_or_404
from .models import Book, Category

def category_books(request, slug):
    """Display books in a specific category."""
    category = get_object_or_404(Category, slug=slug)
    books = get_list_or_404(Book, category=category, in_stock=True)
    return render(request, 'catalog/category_books.html', {
        'category': category,
        'books': books
    })


# catalog/urls.py
urlpatterns = [
    path('category/<slug:slug>/', views.category_books, name='category_books'),
]
```

</details>

### Exercise 3: Search View with GET Parameters

Create a search view that accepts `q` (query), `min_price`, and `max_price` as GET parameters and returns matching books.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.shortcuts import render
from .models import Book

def search(request):
    """Search books with optional filters."""
    query = request.GET.get('q', '').strip()
    min_price = request.GET.get('min_price')
    max_price = request.GET.get('max_price')
    
    # Start with all books
    books = Book.objects.all()
    
    # Apply filters if provided
    if query:
        books = books.filter(title__icontains=query)
    
    if min_price:
        try:
            books = books.filter(price__gte=float(min_price))
        except ValueError:
            pass  # Ignore invalid price
    
    if max_price:
        try:
            books = books.filter(price__lte=float(max_price))
        except ValueError:
            pass
    
    return render(request, 'catalog/search_results.html', {
        'books': books,
        'query': query,
        'min_price': min_price,
        'max_price': max_price,
    })


# catalog/urls.py
urlpatterns = [
    path('search/', views.search, name='search'),
    # Usage: /search/?q=python&min_price=10&max_price=50
]
```

</details>

### Exercise 4: Class-Based View with Pagination

Create a ListView that displays books with pagination (10 per page), ordered by title, and only shows books in stock.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.views.generic import ListView
from .models import Book

class BookListView(ListView):
    """Display paginated list of available books."""
    model = Book
    template_name = 'catalog/book_list.html'
    context_object_name = 'books'
    paginate_by = 10
    ordering = ['title']
    
    def get_queryset(self):
        """Only return books in stock."""
        return Book.objects.filter(in_stock=True).order_by('title')
    
    def get_context_data(self, **kwargs):
        """Add extra context."""
        context = super().get_context_data(**kwargs)
        context['total_available'] = self.get_queryset().count()
        return context


# catalog/urls.py
from .views import BookListView

urlpatterns = [
    path('books/', BookListView.as_view(), name='book_list'),
]
```

</details>

### Exercise 5: CRUD Views for Author

Create complete CRUD views for the Author model using generic class-based views.

<details>
<summary>Click to see solution</summary>

```python
# catalog/views.py
from django.views.generic import ListView, DetailView, CreateView, UpdateView, DeleteView
from django.urls import reverse_lazy
from .models import Author

class AuthorListView(ListView):
    """List all authors."""
    model = Author
    template_name = 'catalog/author_list.html'
    context_object_name = 'authors'
    paginate_by = 20


class AuthorDetailView(DetailView):
    """Display author details and their books."""
    model = Author
    template_name = 'catalog/author_detail.html'
    context_object_name = 'author'
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['books'] = self.object.books.all()
        return context


class AuthorCreateView(CreateView):
    """Create a new author."""
    model = Author
    template_name = 'catalog/author_form.html'
    fields = ['first_name', 'last_name', 'birth_date', 'biography']
    success_url = reverse_lazy('catalog:author_list')


class AuthorUpdateView(UpdateView):
    """Update an existing author."""
    model = Author
    template_name = 'catalog/author_form.html'
    fields = ['first_name', 'last_name', 'birth_date', 'biography']
    
    def get_success_url(self):
        return reverse_lazy('catalog:author_detail', kwargs={'pk': self.object.pk})


class AuthorDeleteView(DeleteView):
    """Delete an author."""
    model = Author
    template_name = 'catalog/author_confirm_delete.html'
    success_url = reverse_lazy('catalog:author_list')


# catalog/urls.py
from .views import (
    AuthorListView, AuthorDetailView, 
    AuthorCreateView, AuthorUpdateView, AuthorDeleteView
)

urlpatterns = [
    path('authors/', AuthorListView.as_view(), name='author_list'),
    path('authors/<int:pk>/', AuthorDetailView.as_view(), name='author_detail'),
    path('authors/new/', AuthorCreateView.as_view(), name='author_create'),
    path('authors/<int:pk>/edit/', AuthorUpdateView.as_view(), name='author_update'),
    path('authors/<int:pk>/delete/', AuthorDeleteView.as_view(), name='author_delete'),
]
```

</details>

---

## Summary

In this notebook, you learned:

- **Function-based views (FBVs)** are simple functions that take a request and return a response
- **HttpRequest** contains all information about the incoming request
- **HttpResponse** (and subclasses) represent the server's response
- **URL configuration** uses `path()` and `include()` to map URLs to views
- **Path converters** (`int`, `str`, `slug`, `uuid`, `path`) capture URL parameters
- **Class-based views (CBVs)** organize view logic using classes
- **Generic views** provide common patterns: ListView, DetailView, CreateView, UpdateView, DeleteView
- **Shortcuts** like `render()`, `redirect()`, `get_object_or_404()` simplify common tasks
- **Decorators** add functionality like `@login_required` and `@require_POST`

## Next Steps

In the next notebook, we'll explore **Django Templates**, where you'll learn how to:
- Create HTML templates with the Django Template Language
- Use template inheritance and includes
- Pass context data to templates
- Work with static files (CSS, JavaScript, images)